<a href="https://colab.research.google.com/github/jlieow/tools/blob/master/reddit_prediction_scrapper_without_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!!!THIS IS A PUBLIC COPY!!!
#!!!PLEASE SAVE A COPY TO YOUR OWN DRIVE!!! GO to File >  Save a copy in drive
#DO NOT SAVE ANYTHING TO THIS COPY!!!
#!!!ANYTHING SAVED HERE IS VISIBLE TO EVERYONE!!!

#Prior to running this in Colab, you need to add 4 files into root/content.
#In the sidebar, there should be a folder icon. Click it to navigate to the directory.
#You should be inside root/content and see 2 folders, 'Up one level' and sample_data.
#In order to add files, click on 'Up one level to navigate into /root. 
#There are 3 dots on the right side of the folder 'content'. Go to 3 dots > New file and add the following files: 
##CLIENT_ID.txt
##SECRET_KEY.txt
##PW.txt
##ARTICLE_ID.txt

#CLIENT_ID.txt and SECRET_KEY.txt values are taken from reddit page https://www.reddit.com/prefs/apps.
#In the page, click on the 'create another app...' button at the bottom of the page. 
#Input the name of your app, select script and include a redirect URI.
#The values value under 'personal use script' and 'secret' should be generated for you.
##Insert the value under 'personal use script' into CLIENT_ID.txt
##Insert the value under 'secret' into SECRET_KEY.txt

#PW.txt is your reddit password
#If you sign in using a third party authenticator like gmail, you would not have registered a password and would need to reset it. 
#You can reset your password either via the login page, or by enabling 'two factor authentication' under the reddit page https://www.reddit.com/settings/privacy.
##Insert the value into PW.txt

#ARTICLE_ID.txt is your article that you are targetting to scrape
##Example from actual URL see ARTICLE_ID after '/comments/':
##https://www.reddit.com/r/TSLALounge/comments/npuzji/friday_predictions_june_4/
##ARTICLE_ID = npuzji
##Insert the value into ARTICLE_ID.txt

###Each file should only contain the value and nothing else
###If done correctly, code should execute without flaw

In [ ]:
#Reading CLIENT_ID, SECRET_KEY, PW and ARTICLE_ID from file in /content
#Preparing CLIENT_ID and SECRET_KEY from reddit page https://www.reddit.com/prefs/apps
with open('CLIENT_ID.txt', 'r') as f:
  CLIENT_ID = f.read()

with open('SECRET_KEY.txt', 'r') as f:
  SECRET_KEY = f.read()

#Preparing PW 
with open('PW.txt', 'r') as f:
  PW = f.read()

#Preparing ARTICLE 
with open('ARTICLE_ID.txt', 'r') as f:
  ARTICLE_ID = f.read()

In [ ]:
#Use CLIENT_ID, SECRET_KEY and PW to get TOKEN
import requests

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

data = {
    'grant_type': 'password',
    'username': 'bullheadedbear',
    'password': PW
}

headers = {'User-Agent': 'MyAPI/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']

In [ ]:
#Insert TOKEN into HTTP header
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}

In [ ]:
#Test Response
#200 = Successful/Authorized
#Response 403 = Forbidden/Unauthorized (Likely due to incorrect TOKEN in HTTP header)
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [ ]:
#Import pandas lib
import pandas as pd

#Get the comment tree for a given Article 
prediction_res = requests.get('https://oauth.reddit.com/r/TSLALounge/comments/' + ARTICLE_ID, headers=headers).json()

prediction_df = pd.DataFrame()

for post in prediction_res[1]['data']['children']:
  prediction_df = prediction_df.append({
      'author': post['data']['author'],
      'body': post['data']['body'],
  }, ignore_index=True)

prediction_df